In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional
import json

from medical_ner import extract_medical_entities
from medical_summary import generate_medical_summary
from sentiment_intent import classify_sentiment
from intent_detection import detect_intent

app = FastAPI(
    title="Medical NLP Pipeline API",
    description="Medical Transcription, Summarization, Sentiment & Intent Analysis",
    version="1.0.0"
)


C:\Users\spatr\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [2]:
class MedicalRequest(BaseModel):
    transcript: str
    patient_dialogue: Optional[str] = None


In [3]:
class MedicalResponse(BaseModel):
    Patient_Name: str
    Symptoms: list
    Diagnosis: str
    Treatment: list
    Current_Status: str
    Prognosis: str
    Sentiment: Optional[str]
    Intent: Optional[str]


In [4]:
@app.post("/analyze", response_model=MedicalResponse)
def analyze_medical_text(request: MedicalRequest):
    # Step 1: NER
    entities = extract_medical_entities(request.transcript)

    # Step 2: Medical Summary
    summary = generate_medical_summary(entities)

    # Step 3: Sentiment & Intent (optional)
    sentiment = None
    intent = None

    if request.patient_dialogue:
        sentiment = classify_sentiment(request.patient_dialogue)
        intent = detect_intent(request.patient_dialogue)

    response = {
        **summary,
        "Sentiment": sentiment,
        "Intent": intent
    }

    return response
